In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bengkulu,-3.8004,102.2655,75.24,91,98,2.59,ID,2021-09-27 22:27:18
1,1,Carnarvon,-24.8667,113.6333,50.07,87,8,8.05,AU,2021-09-27 22:27:19
2,2,Jamestown,42.0970,-79.2353,74.52,60,1,12.66,US,2021-09-27 22:12:46
3,3,Saskylakh,71.9167,114.0833,24.80,91,13,9.95,RU,2021-09-27 22:27:18
4,4,Voyvozh,62.8920,54.9635,32.49,83,92,3.58,RU,2021-09-27 22:27:19


In [3]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Create the heatmap for the meximum temperature. 
# 1. Assign the locations to an array of latitude and longitude pairs
# locations = [latitude, longitude]
# 2. Assign the weights variable to some values 
# Temperatures = # an array of length equal to the locations array length 
# 3. Assign the figure variable to the gmaps.figure() attribute 
# fig = gmaps.figure() 
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute
# and add in the locations.
# heatmap_layer = gmaps.heatmap_layer(locations, weights=temperatures)
# 5. Add the heatmap layer.
# fig.add_layer(heatmap_layer)
# 6. Call the figure to plot the data.
# fig

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bengkulu,-3.8004,102.2655,75.24,91,98,2.59,ID,2021-09-27 22:27:18
2,2,Jamestown,42.0970,-79.2353,74.52,60,1,12.66,US,2021-09-27 22:12:46
5,5,Farafangana,-22.8167,47.8333,70.30,83,57,4.99,MG,2021-09-27 22:27:19
7,7,Souillac,-20.5167,57.5167,71.37,73,40,11.50,MU,2021-09-27 22:17:37
10,10,Sechura,-5.5569,-80.8222,73.58,62,75,15.26,PE,2021-09-27 22:27:20
14,14,Grand Gaube,-20.0064,57.6608,71.38,71,52,11.01,MU,2021-09-27 22:27:21
15,15,Rikitea,-23.1203,-134.9692,73.72,72,22,21.63,PF,2021-09-27 22:27:11
19,19,Georgetown,5.4112,100.3354,78.73,93,20,1.01,MY,2021-09-27 22:26:46
20,20,Callaway,30.1530,-85.5699,85.91,64,1,4.00,US,2021-09-27 22:27:22
22,22,Caravelas,-17.7125,-39.2481,73.62,81,61,8.59,BR,2021-09-27 22:18:26


In [10]:
preferred_cities_df.count()

City_ID       270
City          270
Lat           270
Lng           270
Max Temp      270
Humidity      270
Cloudiness    270
Wind Speed    270
Country       269
Date          270
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Bengkulu,ID,75.24,-3.8004,102.2655,
2,Jamestown,US,74.52,42.0970,-79.2353,
5,Farafangana,MG,70.30,-22.8167,47.8333,
7,Souillac,MU,71.37,-20.5167,57.5167,
10,Sechura,PE,73.58,-5.5569,-80.8222,
14,Grand Gaube,MU,71.38,-20.0064,57.6608,
15,Rikitea,PF,73.72,-23.1203,-134.9692,
19,Georgetown,MY,78.73,5.4112,100.3354,
20,Callaway,US,85.91,30.1530,-85.5699,
22,Caravelas,BR,73.62,-17.7125,-39.2481,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [18]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [19]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [20]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))